## Libraries

In [3]:
# Standard library imports
import os
import sys

# Third party imports
import numpy as np
import buteo as beo
from glob import glob
from tqdm import tqdm
import import_ipynb

# Local imports
from utilities import custom_subplots
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

## Split Train and Test Set

To make sure the test set is completely independant from the train set, we will split the data before making the patches.

In [4]:
# Set the data folder
DATA_FOLDER = os.path.abspath(os.path.join(os.getcwd(), '../data'))

# Set the seeds
np.random.seed(17)
TEST_INDICES = np.random.choice(78, int(78 * 0.2))
VAL_INDICES = TEST_INDICES[:len(TEST_INDICES) // 2]
TEST_INDICES = TEST_INDICES[len(TEST_INDICES) // 2:]

# We will use patch size of 32x32 to reduce the amount of RAM required to run the model
PATCH_SIZE = 32

# With zero offsets, every pixel in the training data would only be sampled once.
# By setting overlaps, we can sample pixel multiple times but in different places in the patches.
N_OFFSETS = 3 # ((0, 0), (8, 8), (16, 16), (24, 24))

# Patch lists
training_labels = []
training_s1 = []
training_s2 = []
training_dem = []

val_labels = []
val_s1 = []
val_s2 = []
val_dem = []

testing_labels = []
testing_s1 = []
testing_s2 = []
testing_dem = []

labels_paths = sorted(glob(os.path.join(DATA_FOLDER, 'CCAI_FLOODS_DATA/label_*.tif')))
s1_paths = sorted(glob(os.path.join(DATA_FOLDER, 'CCAI_FLOODS_DATA/s1_*.tif')))
s2_paths = sorted(glob(os.path.join(DATA_FOLDER, 'CCAI_FLOODS_DATA/s2_*.tif')))
dem_paths = sorted(glob(os.path.join(DATA_FOLDER, 'CCAI_FLOODS_DATA/dem_*.tif')))

# Read and order the tiles in a temporary folder
for image in tqdm(
  zip(labels_paths, s1_paths, s2_paths, dem_paths),
  total=len(labels_paths),
  ncols=120
):
  label_path, s1_path, s2_path, dem_path = image

  # Get the name and number of the patches
  label_name = os.path.splitext(os.path.basename(label_path))[0]
  img_idx = int(label_name.split('_')[1])

  # Get the data from the tiles
  arr_label = beo.raster_to_array(label_path, filled=True, fill_value=0.0)

  # Handle any potential errors
  np.clip(arr_label, 0.0, 100.0, out=arr_label)
  arr_label[np.isnan(arr_label)] = 0.0

  # Read the tiles
  arr_s1 = beo.raster_to_array(s1_path, filled=True, fill_value=0.0)
  arr_s2 = beo.raster_to_array(s2_path, filled=True, fill_value=0.0)
  arr_dem = beo.raster_to_array(dem_path, filled=True, fill_value=0.0)
  
  # Generate the patches
  label_patches = beo.array_to_patches(arr_label, PATCH_SIZE, n_offsets=N_OFFSETS)
  s1_patches = beo.array_to_patches(arr_s1, PATCH_SIZE, n_offsets=N_OFFSETS)
  s2_patches = beo.array_to_patches(arr_s2, PATCH_SIZE, n_offsets=N_OFFSETS)
  dem_patches = beo.array_to_patches(arr_dem, PATCH_SIZE, n_offsets=N_OFFSETS)

  # Sanity check to ensure that the right images were chosen
  assert label_patches.shape[0:3] == s1_patches.shape[0:3] == s2_patches.shape[0:3] == dem_patches.shape[0:3], \
    f'Patch shapes do not match for image {img_idx}'
  
  if img_idx in TEST_INDICES:
    testing_labels.append(label_patches)
    testing_s1.append(s1_patches)
    testing_s2.append(s2_patches)
    testing_dem.append(dem_patches)
  elif img_idx in VAL_INDICES:
    val_labels.append(label_patches)
    val_s1.append(s1_patches)
    val_s2.append(s2_patches)
    val_dem.append(dem_patches)
  else:
    training_labels.append(label_patches)
    training_s1.append(s1_patches)
    training_s2.append(s2_patches)
    training_dem.append(dem_patches)

# Merge the patches back together
training_labels = np.concatenate(training_labels, axis=0)
training_s1 = np.concatenate(training_s1, axis=0)
training_s2 = np.concatenate(training_s2, axis=0)
training_dem = np.concatenate(training_dem, axis=0)

val_labels = np.concatenate(val_labels, axis=0)
val_s1 = np.concatenate(val_s1, axis=0)
val_s2 = np.concatenate(val_s2, axis=0)
val_dem = np.concatenate(val_dem, axis=0)

testing_labels = np.concatenate(testing_labels, axis=0)
testing_s1 = np.concatenate(testing_s1, axis=0)
testing_s2 = np.concatenate(testing_s2, axis=0)
testing_dem = np.concatenate(testing_dem, axis=0)

print('Training Labels: ', training_labels.shape, training_s1.shape, training_s2.shape, training_dem.shape)
print('Test Indices: ', TEST_INDICES)
print('Testing Labels: ', testing_labels.shape, testing_s1.shape, testing_s2.shape, testing_dem.shape)
print('Validation Indices: ', VAL_INDICES)
print('Validation Labels: ', val_labels.shape, val_s1.shape, val_s2.shape, val_dem.shape)



100%|███████████████████████████████████████████████████████████████████████████████████| 78/78 [00:18<00:00,  4.11it/s]


Training Labels:  (28120, 32, 32, 1) (28120, 32, 32, 2) (28120, 32, 32, 9) (28120, 32, 32, 4)
Test Indices:  [68 39 44  7  1 17 41 56]
Testing Labels:  (1392, 32, 32, 1) (1392, 32, 32, 2) (1392, 32, 32, 9) (1392, 32, 32, 4)
Validation Indices:  [15  6 22 57 45 22 31]
Validation Labels:  (3292, 32, 32, 1) (3292, 32, 32, 2) (3292, 32, 32, 9) (3292, 32, 32, 4)


## Normalisation

**Sentinel-1**: The current data is expressed in dB. The data is first clipped to the range (-5 dB, 35 dB). Afterwards each value is normalised to (0, 1) by subtracting the mean and dividing by the standard deviation.

**Sentinel-2**: The Sentinel-2 images are uint16 with values ranging from 0 to 65,535 which are called digital numbers (DN). The physical value derived from this is the reflectance defined as DN/10,000. The reflectance is usually between 0-1 but can be higher due to surface or cloud effects. By convention, Sentinel-2 data is clipped to the range (0, 10,000). Afterwards it is normalised to (0, 1) as well.

**CopDEM**: The channels in the DEM are already normalised. The elevation (4th channel) is normalised with respect to the highest point on the Earth, Mt. Everest. The slope (3rd band) is also normalised with respect to a slope of 90&deg;. The first two bands contain the orientation of the slope. The orientation is a cyclical variable (0&deg;-360&deg;), so we need two channels to encode and normalise this information. This can be done by storing the sine of the orientation in the first band and the cosine in the second band to keep the range between (0, 1). The final result is:

$channel 1 = [sin(orientation) + 1]/2$

$channel 2 = [cos(orientation) + 1]/2$

In [5]:
s1_train, _statdict = beo.scaler_truncate(training_s1, -35.0, 5.0)
val_train, _statdict = beo.scaler_truncate(val_s1, -35.0, 5.0)
s1_test, _statdict = beo.scaler_truncate(testing_s1, -35.0, 5.0)

s2_train, _statdict = beo.scaler_truncate(training_s2, 0.0, 10000.0)
val_s2, _statdict = beo.scaler_truncate(val_s2, 0.0, 10000.0)
s2_test, _statdict = beo.scaler_truncate(testing_s2, 0.0, 10000.0)

# If labels are not already float32, convert them to float32
train_labels = training_labels.astype(np.float32, copy=False)
val_labels = val_labels.astype(np.float32, copy=False)
test_labels = testing_labels.astype(np.float32, copy=False)

Let's save all our work to disk as NumPy arrays, so that at any moment in time, we could reload the data again and use it to train the model. When loading the data again, the data can be kept on disk, and only necessary bits can be loaded into memory upon access.

In [ ]:
np.savez_compressed(os.path.join(DATA_FOLDER, 'train.npz'), x_s1=s1_train, x_s2=s2_train, x_dem=training_dem, y=train_labels)
np.savez_compressed(os.path.join(DATA_FOLDER, 'val.npz'), x_s1=val_s1, x_s2=val_s2, x_dem=val_dem, y=val_labels)
np.savez_compressed(os.path.join(DATA_FOLDER, 'test.npz'), x_s1=s1_test, x_s2=s2_test, x_dem=testing_dem, y=test_labels)

# Clean memory
del s1_train, s2_train, training_dem, train_labels
del val_s1, val_s2, val_dem, val_labels

del s1_test, s2_test, testing_dem, test_labels